In [42]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [43]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [44]:
application_df = application_df.drop(columns=["STATUS","SPECIAL_CONSIDERATIONS"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [45]:
# Determine the number of unique values in each column.
application_categ = application_df.dtypes[application_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
application_df[application_categ].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [46]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_count[app_type_count < 550].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [47]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1880].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [48]:
# Generate our categorical variable lists
category = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
category

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [49]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[category]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(category)
encode_df.head()

/Users/bneves/opt/anaconda3/envs/mlenv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(columns=category)


application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns="IS_SUCCESSFUL").values
y = application_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## 1st attempt: Removing 2 columns and changing the amount of neurons

In [17]:
len(X_train_scaled[0])

39

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 52
hidden_nodes_layer2 = 26

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 52)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 26)                1378      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 27        
Total params: 3,485
Trainable params: 3,485
Non-trainable params: 0
_________________________________________________________________


2022-08-29 15:27:54.288884: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-29 15:27:54.290144: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Checkpoint path and filenames
os.makedirs("opt1_checkpoints/",exist_ok=True)
checkpoint_path = "opt1_checkpoints/weights.{epoch:02d}.hdf5"

In [21]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

2022-08-29 15:30:14.008100: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
804/804 [==============================] - 1s 439us/step - loss: 0.5985 - accuracy: 0.6954
Epoch 2/100
804/804 [==============================] - 0s 415us/step - loss: 0.5596 - accuracy: 0.7258
Epoch 3/100
804/804 [==============================] - 0s 423us/step - loss: 0.5564 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 0s 437us/step - loss: 0.5497 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 0s 421us/step - loss: 0.5506 - accuracy: 0.7284

Epoch 00005: saving model to opt1_checkpoints/weights.05.hdf5
Epoch 6/100
804/804 [==============================] - 0s 424us/step - loss: 0.5499 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 0s 434us/step - loss: 0.5556 - accuracy: 0.7265
Epoch 8/100
804/804 [==============================] - 0s 416us/step - loss: 0.5464 - accuracy: 0.7335
Epoch 9/100
804/804 [==============================] - 0s 414us/step - loss: 0.5496 - accuracy: 0.7281
Epoch 10/1

804/804 [==============================] - 0s 553us/step - loss: 0.5415 - accuracy: 0.7327
Epoch 72/100
804/804 [==============================] - 1s 677us/step - loss: 0.5439 - accuracy: 0.7328
Epoch 73/100
804/804 [==============================] - 0s 412us/step - loss: 0.5377 - accuracy: 0.7380
Epoch 74/100
804/804 [==============================] - 0s 410us/step - loss: 0.5408 - accuracy: 0.7349
Epoch 75/100
804/804 [==============================] - 0s 411us/step - loss: 0.5439 - accuracy: 0.73320s - loss: 0.5442 - accuracy: 0.73

Epoch 00075: saving model to opt1_checkpoints/weights.75.hdf5
Epoch 76/100
804/804 [==============================] - 0s 413us/step - loss: 0.5373 - accuracy: 0.7372
Epoch 77/100
804/804 [==============================] - 0s 412us/step - loss: 0.5386 - accuracy: 0.7369
Epoch 78/100
804/804 [==============================] - 0s 414us/step - loss: 0.5384 - accuracy: 0.7387
Epoch 79/100
804/804 [==============================] - 0s 413us/step - loss: 0.5353

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5502 - accuracy: 0.7305
Loss: 0.5502233505249023, Accuracy: 0.7304956316947937


In [24]:
# Export our model to HDF5 file
nn.save("opt1_alphabet_soup.h5")

## SECOND ATTEMPT: Same pattern, different Activation functions

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 52
hidden_nodes_layer2 = 26

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 52)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 26)                1378      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 27        
Total params: 3,485
Trainable params: 3,485
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:
# Checkpoint path and filenames
os.makedirs("opt2_checkpoints/",exist_ok=True)
checkpoint_path = "opt2_checkpoints/weights.{epoch:02d}.hdf5"

In [39]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [40]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=120,callbacks=[cp_callback])

Epoch 1/120
804/804 [==============================] - 1s 448us/step - loss: 0.5798 - accuracy: 0.7091
Epoch 2/120
804/804 [==============================] - 0s 418us/step - loss: 0.5522 - accuracy: 0.7294
Epoch 3/120
804/804 [==============================] - 0s 416us/step - loss: 0.5490 - accuracy: 0.7312
Epoch 4/120
804/804 [==============================] - 0s 427us/step - loss: 0.5508 - accuracy: 0.7280
Epoch 5/120
804/804 [==============================] - 0s 433us/step - loss: 0.5500 - accuracy: 0.7321

Epoch 00005: saving model to opt2_checkpoints/weights.05.hdf5
Epoch 6/120
804/804 [==============================] - 0s 423us/step - loss: 0.5491 - accuracy: 0.7333
Epoch 7/120
804/804 [==============================] - 0s 418us/step - loss: 0.5504 - accuracy: 0.7281
Epoch 8/120
804/804 [==============================] - 0s 414us/step - loss: 0.5450 - accuracy: 0.7333
Epoch 9/120
804/804 [==============================] - 0s 429us/step - loss: 0.5469 - accuracy: 0.7316
Epoch 10/1

804/804 [==============================] - 0s 418us/step - loss: 0.5369 - accuracy: 0.73690s - loss: 0.5376 - accuracy: 0.
Epoch 72/120
804/804 [==============================] - 0s 412us/step - loss: 0.5357 - accuracy: 0.7375
Epoch 73/120
804/804 [==============================] - 0s 412us/step - loss: 0.5316 - accuracy: 0.7428
Epoch 74/120
804/804 [==============================] - 0s 411us/step - loss: 0.5347 - accuracy: 0.7386
Epoch 75/120
804/804 [==============================] - 0s 414us/step - loss: 0.5373 - accuracy: 0.7359

Epoch 00075: saving model to opt2_checkpoints/weights.75.hdf5
Epoch 76/120
804/804 [==============================] - 0s 413us/step - loss: 0.5387 - accuracy: 0.7370
Epoch 77/120
804/804 [==============================] - 0s 420us/step - loss: 0.5393 - accuracy: 0.7356
Epoch 78/120
804/804 [==============================] - 0s 417us/step - loss: 0.5309 - accuracy: 0.7431
Epoch 79/120
804/804 [==============================] - 0s 415us/step - loss: 0.5336 -

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("opt2_alphabet_soup.h5")

268/268 - 0s - loss: 0.5654 - accuracy: 0.7275
Loss: 0.5653541684150696, Accuracy: 0.7274635434150696


## Third Attempt: Changing Epoch number, neurons, and activation functions

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 60)                2400      
_________________________________________________________________
dense_10 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 31        
Total params: 4,261
Trainable params: 4,261
Non-trainable params: 0
_________________________________________________________________


In [55]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Checkpoint path and filenames
os.makedirs("opt3_checkpoints/",exist_ok=True)
checkpoint_path = "opt3_checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [57]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 478us/step - loss: 0.5811 - accuracy: 0.5331
Epoch 2/150
804/804 [==============================] - 0s 464us/step - loss: 0.5523 - accuracy: 0.5360
Epoch 3/150
804/804 [==============================] - 0s 473us/step - loss: 0.5544 - accuracy: 0.5325
Epoch 4/150
804/804 [==============================] - 0s 474us/step - loss: 0.5518 - accuracy: 0.5299
Epoch 5/150
804/804 [==============================] - 0s 475us/step - loss: 0.5481 - accuracy: 0.5308

Epoch 00005: saving model to opt3_checkpoints/weights.05.hdf5
Epoch 6/150
804/804 [==============================] - 0s 468us/step - loss: 0.5513 - accuracy: 0.5319
Epoch 7/150
804/804 [==============================] - 0s 462us/step - loss: 0.5442 - accuracy: 0.5276
Epoch 8/150
804/804 [==============================] - 0s 462us/step - loss: 0.5460 - accuracy: 0.5277
Epoch 9/150
804/804 [==============================] - 0s 462us/step - loss: 0.5476 - accuracy: 0.5305
Epoch 10/1

804/804 [==============================] - 0s 465us/step - loss: 0.5344 - accuracy: 0.5303
Epoch 72/150
804/804 [==============================] - 0s 459us/step - loss: 0.5367 - accuracy: 0.5327
Epoch 73/150
804/804 [==============================] - 0s 459us/step - loss: 0.5323 - accuracy: 0.5284
Epoch 74/150
804/804 [==============================] - 0s 461us/step - loss: 0.5362 - accuracy: 0.5310
Epoch 75/150
804/804 [==============================] - 0s 460us/step - loss: 0.5349 - accuracy: 0.5274

Epoch 00075: saving model to opt3_checkpoints/weights.75.hdf5
Epoch 76/150
804/804 [==============================] - 0s 466us/step - loss: 0.5255 - accuracy: 0.5244
Epoch 77/150
804/804 [==============================] - 0s 475us/step - loss: 0.5367 - accuracy: 0.5306
Epoch 78/150
804/804 [==============================] - 0s 469us/step - loss: 0.5391 - accuracy: 0.5273
Epoch 79/150
804/804 [==============================] - 0s 481us/step - loss: 0.5373 - accuracy: 0.5333
Epoch 80/150
8

804/804 [==============================] - 0s 462us/step - loss: 0.5301 - accuracy: 0.5301
Epoch 142/150
804/804 [==============================] - 0s 457us/step - loss: 0.5293 - accuracy: 0.5346
Epoch 143/150
804/804 [==============================] - 0s 461us/step - loss: 0.5314 - accuracy: 0.5341
Epoch 144/150
804/804 [==============================] - 0s 460us/step - loss: 0.5316 - accuracy: 0.5268
Epoch 145/150
804/804 [==============================] - 0s 460us/step - loss: 0.5324 - accuracy: 0.5266

Epoch 00145: saving model to opt3_checkpoints/weights.145.hdf5
Epoch 146/150
804/804 [==============================] - 0s 459us/step - loss: 0.5295 - accuracy: 0.5314
Epoch 147/150
804/804 [==============================] - 0s 460us/step - loss: 0.5277 - accuracy: 0.5341
Epoch 148/150
804/804 [==============================] - 0s 463us/step - loss: 0.5317 - accuracy: 0.5319
Epoch 149/150
804/804 [==============================] - 0s 461us/step - loss: 0.5312 - accuracy: 0.5352
Epoch

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("opt3_alphabet_soup.h5")

268/268 - 0s - loss: 0.5628 - accuracy: 0.5394
Loss: 0.5627564191818237, Accuracy: 0.5393586158752441
